In [1]:
import pandas as pd

# read tweet file containing finbert_score
tweets = pd.read_csv("../2_data/finbert_scored_tweets.csv", parse_dates=["created_at"])

# keep ddmmyyyy only
tweets["Date"] = tweets["created_at"].dt.date

# day, mean
daily_sent = (
    tweets
      .groupby("Date")["finbert_score"]
      .mean()
      .reset_index()
      .rename(columns={"finbert_score": "daily_finbert"})
)

# transfer Date to datetime type
daily_sent["created_at"] = pd.to_datetime(daily_sent["Date"])
daily_sent = daily_sent[["created_at", "daily_finbert"]]

print(daily_sent.head())

/var/folders/rn/lc8n68_d7ld6yymqc_lsnrr00000gn/T/ipykernel_37417/3451837175.py:4: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets = pd.read_csv("../2_data/finbert_scored_tweets.csv", parse_dates=["created_at"])


  created_at  daily_finbert
0 2013-04-11       0.105690
1 2013-04-12       0.208949
2 2013-04-13       0.709719
3 2013-04-14       0.357927
4 2013-04-15      -0.107966


- VIX and Price

In [2]:
# read vix data
vix = pd.read_csv(
    "../2_data/vix_data.csv",
    header=[0,1],      # header
    index_col=0,       # take date as index
    parse_dates=True
)
vix_close = vix[("Close","^VIX")].to_frame(name="vix_close")

# read nvidia price data
price = pd.read_csv(
    "../2_data/nvda_price_volume.csv",
    header=[0,1],
    index_col=0,
    parse_dates=True
)
nvda_close = price[("Close","NVDA")].to_frame(name="nvda_close")
# return (each day)
nvda_close["nvda_return"] = nvda_close["nvda_close"].pct_change()
nvda_close = nvda_close.dropna(subset=["nvda_return"])

- Alignment

In [4]:
# change index name of nvda_close and vix_close to Datetime
vix_close.index.name = "created_at"
nvda_close.index.name = "created_at"

# merge
merged = (
    daily_sent
      .set_index("created_at")
      .join(nvda_close[["nvda_close", "nvda_return"]], how="inner")
      .join(vix_close, how="inner")
      .reset_index()
)

print(merged.head())
print(f"After alighment there are data of {len(merged)} days")

merged.to_csv("../2_data/sentiment_price_vix_aligned.csv", index=False)

  created_at  daily_finbert  nvda_close  nvda_return  vix_close
0 2022-01-04       0.135383   29.238680    -0.027589  16.910000
1 2022-01-05       0.124786   27.555632    -0.057562  19.730000
2 2022-01-06       0.127680   28.128624     0.020794  19.610001
3 2022-01-07       0.118334   27.199259    -0.033040  18.760000
4 2022-01-10       0.125386   27.351990     0.005615  19.400000
After alighment there are data of 38 days


Not bad, cuz 78 days in total in our dataset